In [1]:
%matplotlib ipympl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import pickle # to access dataframe faster than csv
import glob, re
import os
import csv
from pathlib import Path

### Get LCLids to ignore due to limited data

In [2]:
file = open('preprocessed_dfs/batch_1_durations.pkl', 'rb')
df_b1 = pickle.load(file)

file = open('preprocessed_dfs/batch_2_durations.pkl', 'rb')
df_b2 = pickle.load(file)

file = open('preprocessed_dfs/batch_3_durations.pkl', 'rb')
df_b3 = pickle.load(file)

file = open('preprocessed_dfs/batch_4_durations.pkl', 'rb')
df_b4 = pickle.load(file)

windows_df_all = pd.concat([df_b1, df_b2, df_b3, df_b4])

windows_df_all.sort_values(by=['Duration'], inplace = True)
windows_ri_all = windows_df_all.reset_index()
windows_ri_all['LCLid'] = windows_ri_all['LCLid'].apply(lambda x: int(re.sub('\D', '', x)))
windows_ri_all.drop(columns = ['index'], inplace = True)
# drop duplicates
print('Number of duplicates: ', windows_ri_all.duplicated(subset=['LCLid']).sum())
print('because of concatenating 4 batches. 1 id common between every 2 batches')
windows_ri_uq = windows_ri_all.drop_duplicates(subset = ['LCLid']).copy()

Number of duplicates:  3
because of concatenating 4 batches. 1 id common between every 2 batches


In [3]:
# get the list of ignored MACid with zero duration.
ignored_ids=windows_ri_all[windows_ri_all['Duration']==pd.Timedelta(0)]['LCLid'] # ids with zero duration
ignored_MACs=[f'MAC{lclid:06d}' for lclid in ignored_ids] # 

### Clean data 

In [4]:
filenames = sorted(glob.glob('uk-smart-meter-data/LCL-June2015v2_*.csv'))
dest_path='uk_smart_meter_cleaned_imputed'

In [5]:
tot=0
tot_dropped=0
stdortou_dict={}
if False: # make this true for first time
    for filepath in filenames:
        filename = filepath.split('/')[-1].split('\\')[-1].split('.')[0] # uk-smart-meter-data\LCL-June2015v2_0.csv >> LCL-June2015v2_0
        data = pd.read_csv(filepath) 
        data['DateTime']=pd.to_datetime(data['DateTime'])
        data['minutes'] = data['DateTime'].dt.minute
        ## Remove readings not at exact 30 minute intervals.
        filtered_data=data[(data['minutes']==0) | (data['minutes']==30)].drop(columns=['minutes'])
        
        ## Remove duplicates
        filtered_data=filtered_data.drop_duplicates(subset=['DateTime', 'LCLid'], keep='last',inplace=False)
        
        ## Remove data outside 2012-2013
        filtered_data=filtered_data[(filtered_data['DateTime'].dt.year >= 2012) & (filtered_data['DateTime'].dt.year <= 2013)]
        
        ## Ignore LCLids with limited data # remove the rows with LCLid duration 0
        filtered_data=filtered_data[~filtered_data['LCLid'].isin(ignored_MACs)]
        
        for _, row in filtered_data.drop_duplicates(subset=['LCLid']).iterrows():
            lclid = row['LCLid']
            stdortou = row['stdorToU']
            if lclid not in stdortou_dict:
                stdortou_dict[lclid] = stdortou
        
        
        filtered_data=filtered_data.drop(columns=['stdorToU'])
        ## Save to CSV
        filtered_data.reset_index(drop=True,inplace=True)
        filtered_data.to_csv(os.path.join(dest_path,f"{filename}_cleaned"),index=False)
    stdortou_df=pd.DataFrame(list(stdortou_dict.items()),columns=['LCLid','stdorToU'])
    stdortou_df.to_csv(os.path.join(dest_path,'stdorTou_mapping.csv'),index=False)

### Aggregate over each hour

In [6]:
def get_filenum(filename):
    match=re.search(r'June2015v2_(\d+)_cleaned',filename)
    return int(match.group(1) if match else float('inf'))

In [7]:
# instead of sorting 0,1,100,101,102,... 167, 2, 3, 4, ... i.e., lexicographically, sort numerically 0,1,2,3,... 167
filenames = sorted(glob.glob('uk_smart_meter_cleaned_imputed/LCL-June2015v2_*'))
filenames=sorted(filenames,key=get_filenum)

In [8]:
def process_chunk(chunk_data, prior_data=None, final_chunk=False):
    chunk_data['DateTime']=pd.to_datetime(chunk_data['DateTime'])
    
    if prior_data is not None:
        chunk_data=pd.concat([prior_data,chunk_data])
        
    if ~final_chunk:
        last_id=chunk_data['LCLid'].unique()[-1]
        prior_data=chunk_data[chunk_data['LCLid']==last_id]
        chunk_data=chunk_data[chunk_data['LCLid']!=last_id]
        
    pivoted_chunk = chunk_data.pivot(index='DateTime', columns='LCLid', values='KWH/hh (per half hour) ')
    aggregated_chunk = pivoted_chunk.resample('1h').sum(min_count=1)
    return prior_data,aggregated_chunk
    

In [9]:
if False: # true to create chunks in your local machine
    dest_path="uk_smart_meter_aggregated"
    chunk_size = 21
    data = pd.DataFrame()
    data_dict={}
    chunk_num=1
    for chunk_start in range(0,len(filenames),chunk_size):
        chunk_data=pd.DataFrame()
        prior_data=None
        final_chunk=False
        for filename in filenames[chunk_start:chunk_start+chunk_size]:
            #print(filename)
            chunk_data=pd.concat([chunk_data,pd.read_csv(filename)])
        if len(filenames)<=chunk_start+chunk_size:
            final_chunk=True
        prior_data,agg_chunk=process_chunk(chunk_data,prior_data,final_chunk)
        agg_chunk.to_csv(os.path.join(dest_path,f'aggregated_chunk_{chunk_num}.csv'))
        print(f"Chunk {chunk_num} saved.")
        chunk_num+=1


### Define batch times

In [10]:
def blocked_cross_validation(df,train_months=3,test_months=1,overlap_months=0):
    """
    return: list of tuples containing (train_start_date, train_end_date, test_start_date, test_end_date) for year 2012-2013
    """
    blocks=[]
    start_date=df['DateTime'].iloc[0]
    end_date=df['DateTime'].iloc[-1]
    current_train_start=start_date
    while current_train_start+pd.DateOffset(months=train_months+test_months)<=end_date:
        train_end=current_train_start+pd.DateOffset(months=train_months)-pd.Timedelta(hours=1)
        test_start=train_end+pd.Timedelta(hours=1)
        test_end = test_start + pd.DateOffset(months=test_months) - pd.Timedelta(hours=1)
        blocks.append((current_train_start, train_end, test_start, test_end))
        current_train_start = current_train_start + pd.DateOffset(months=train_months + test_months - overlap_months)
    return blocks

In [11]:
df = pd.DataFrame({
    'DateTime': pd.date_range(start='2012-01-01 00:00:00', end='2013-12-31 23:00:00', freq='1h')
})
train_months=3
test_months=1
overlap_months=0
cv_blocks=blocked_cross_validation(df,train_months,test_months,overlap_months) 

### Concatenate the chunk data into a singe dataframe

In [37]:
agg_filepaths = sorted(glob.glob('uk_smart_meter_aggregated/aggregated_chunk_*.csv'))

if False: # Concatenate all chunks into one cookie
    df_agg = pd.DataFrame()
    for agg_filepath in agg_filepaths:
        df_agg = pd.concat([df_agg, pd.read_csv(agg_filepath, index_col = 'DateTime')], axis = 0) # concatenate with respect to DateTime column
    df_agg = df_agg.apply(pd.to_numeric, downcast='float')
    df_agg.info()
    # pickle the file to save time
    df_agg.to_pickle('uk_smart_meter_aggregated/df_agg.pkl')
else:
    file = open('uk_smart_meter_aggregated/df_agg.pkl','rb')
    df_agg = pickle.load(file)



### What are the LCLids in the aggregated data?

In [38]:
# rename df_agg column names; ignore 'MAC0'
if True:
    numeric_columns = [int(re.search(r'\d+', item).group()) for item in df_agg.columns.values]
    list_lclids_agg = np.array(numeric_columns)

[2 3 4 6 7]


In [39]:
# window data for aggregated LCLids; The ids only for the year 2012-2013 
windows_ri_agg = windows_ri_uq[windows_ri_uq['LCLid'].isin(list_lclids_agg)].copy()
if windows_ri_agg.shape[0] == list_lclids_agg.shape[0]:
    print('Aggregated LCLids extracted from the windows data.')

Aggregated LCLids extracted from the windows data.


### Generate similarity matrix for this LCLids using k-nearest neigbour and display the graph.

In [40]:
# given dataframe
windows_ri_agg

,LCLid,Enabled At,Disabled At,Duration
5,2534,2012-07-10 00:30:00,2012-09-23 03:00:00,75 days 02:30:00
6,3346,2012-09-26 10:30:00,2012-12-22 00:00:00,86 days 13:30:00
7,3353,2012-09-26 11:00:00,2013-01-10 00:00:00,105 days 13:00:00
8,3554,2012-10-08 12:30:00,2013-01-24 00:00:00,107 days 11:30:00
9,3559,2013-10-29 18:23:02,2014-02-28 00:00:00,121 days 05:36:58
...,...,...,...,...
5564,150,2011-11-23 10:00:00,2014-02-28 00:00:00,827 days 14:00:00
5565,149,2011-11-23 10:00:00,2014-02-28 00:00:00,827 days 14:00:00
5566,148,2011-11-23 10:00:00,2014-02-28 00:00:00,827 days 14:00:00
5567,147,2011-11-23 10:00:00,2014-02-28 00:00:00,827 days 14:00:00
